import pandas as pd

df = pd.read_csv("CustomerChurn.csv")
df.head()

In [4]:
df.info()
df.describe()

NameError: name 'df' is not defined

In [5]:
df.isnull().sum()

LoyaltyID            0
Customer ID          0
Senior Citizen       0
Partner              0
Dependents           0
Tenure               0
Phone Service        0
Multiple Lines       0
Internet Service     0
Online Security      0
Online Backup        0
Device Protection    0
Tech Support         0
Streaming TV         0
Streaming Movies     0
Contract             0
Paperless Billing    0
Payment Method       0
Monthly Charges      0
Total Charges        0
Churn                0
dtype: int64

In [6]:
df.select_dtypes(include="object").columns

Index(['Customer ID', 'Senior Citizen', 'Partner', 'Dependents',
       'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Payment Method', 'Total Charges', 'Churn'],
      dtype='object')

In [2]:
y = df['Churn']
x = df.drop(columns=['Churn', 'Customer ID'])
x = pd.get_dummies(x, drop_first=True)
from sklearn.model_selection import train_test_split
y = y.str.strip()       
y = y.str.lower()
y = y.map({'no': 0, 'yes': 1})
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train) 

NameError: name 'df' is not defined

In [13]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8211497515968772
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.68      0.60      0.64       373

    accuracy                           0.82      1409
   macro avg       0.77      0.75      0.76      1409
weighted avg       0.82      0.82      0.82      1409



In [1]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(
        max_iter=5000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
y_prob = pipeline.predict_proba(x_test)[:, 1]



from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)

plt.plot(fpr, tpr)
plt.title(f"ROC Curve (AUC = {auc:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

print(set(y_test))
print(y_prob.shape)
print(type(y_prob))

NameError: name 'x_train' is not defined

In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline, x_train, y_train, cv=5, scoring='roc_auc')
print("Mean AUC:", scores.mean())


Mean AUC: 0.7935455837492762


In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
}

grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)
import joblib
joblib.dump(grid.best_params_, "logistic_model.pkl")

Best parameters: {'model__C': 0.01}


['logistic_model.pkl']

In [19]:
best_model = grid.best_estimator_

import pandas as pd

coefficients = best_model.named_steps['model'].coef_[0]
feature_names = x_train.columns

importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

importance = importance.sort_values(by='Coefficient', ascending=False)

print(importance.head(10))

                              Feature  Coefficient
9        Internet Service_Fiber optic     0.293431
27    Payment Method_Electronic check     0.254100
25              Paperless Billing_Yes     0.196470
1638               Total Charges_20.2     0.142545
3                  Senior Citizen_Yes     0.121672
2                     Monthly Charges     0.120037
1644               Total Charges_20.5     0.110823
5755              Total Charges_740.3     0.109236
1520               Total Charges_19.9     0.105551
1521              Total Charges_19.95     0.103503


In [20]:
importance['Abs_Coefficient'] = importance['Coefficient'].abs()
importance = importance.sort_values(by='Abs_Coefficient', ascending=False)
print(importance.head(10))

                              Feature  Coefficient  Abs_Coefficient
1                              Tenure    -0.440143         0.440143
24                  Contract_Two year    -0.300775         0.300775
9        Internet Service_Fiber optic     0.293431         0.293431
27    Payment Method_Electronic check     0.254100         0.254100
23                  Contract_One year    -0.241944         0.241944
12                Online Security_Yes    -0.224758         0.224758
18                   Tech Support_Yes    -0.220608         0.220608
25              Paperless Billing_Yes     0.196470         0.196470
14                  Online Backup_Yes    -0.170917         0.170917
1638               Total Charges_20.2     0.142545         0.142545


B

# 